# 4. 작사가 인공지능 만들기

### 학습과정

1. 데이터 다운로드
2. 데이터 읽어오기
3. 데이터 정제
4. 평가 데이터셋 분리
5. 인공지능 만들기

### 데이터 읽어오기

In [1]:
import re
import numpy as np
import tensorflow as tf
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list=  glob.glob(txt_file_path)

raw_corpus = []

#여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()  # 텍스트를 라인 단위로 끊어서 읽어오기
        raw_corpus.extend(raw)
        
print('데이터의 크기:', len(raw_corpus))
print('Examples:\n', raw_corpus[:30])

데이터의 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?", 'It goes like this', 'The fourth, the fifth', 'The minor fall, the major lift', 'The baffled king composing Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah Your faith was strong but you needed proof', 'You saw her bathing on the roof', 'Her beauty and the moonlight overthrew her', 'She tied you', 'To a kitchen chair', 'She broke your throne, and she cut your hair', 'And from your lips she drew the Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', 'Hallelujah You say I took the name in vain', "I don't even know the name", "But if I did, well really, what's it to you?", "There's a blaze of light", 'In every word', "It doesn't matter which you heard", 'The holy or the broken Hallelujah Hallelujah', 'Hallelujah', 'Hallelujah', "Hallelujah I did my best, it wasn't much", "I couldn't feel, so I tried to touch", 

### 데이터 정제하기

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   #길이가 0인 문장은 건너뜁니다.
    #if sentence[0] == '[': continue    # 문장 처음이 '['인 문장은 건너뜁니다.
    #if sentence[-1] == ']': continue   # 문장 끝이 ']'d인 문장은 건너뜁니다.
    if idx > 30: break   #문장 30개만 확인
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof
You saw her bathing on the roof
Her beauty and the moonlight overthrew her
She tied you
To a kitchen chair
She broke your throne, and she cut your hair
And from your lips she drew the Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah You say I took the name in vain
I don't even know the name
But if I did, well really, what's it to you?
There's a blaze of light
In every word
It doesn't matter which you heard
The holy or the broken Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah I did my best, it wasn't much
I couldn't feel, so I tried to touch
I've told the truth, I didn't come to fool you
And even though


토큰화

    1. 문장 부호 양쪽에 공백을 추가
    2. 모든 문자들을 소문자로 변환
    3. 특수문자들은 모두 제거

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()    # 소문자로 바꾸고 양족 공백을 삭제
    
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)     # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)             # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)   # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    
    sentence = sentence.strip()
    
    sentence = '<start> ' + sentence + ' <end>'       # 문자 앞뒤로 <start>와 <end> 를 단어처럼 붙여줌
    
    if "verse" in sentence:
        sentence = sentence.replace("verse", "")
    if "chorus" in sentence:
        sentence = sentence.replace("chorus", "")
        
    return sentence

In [4]:
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    a0 = preprocess_sentence(sentence)
    if len(a0.split()) > 15: continue
    corpus.append(a0)

In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있음. 이번에는 사용하지 안함
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 <unk> 처리
    )
    tokenizer.fit_on_texts(corpus)   # 구축한 corpus로부터 Tokenizer가 사전을 자동구축

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환.
   
    for num in tensor:
        if len(num) >= 29:
            tensor = np.delete(tensor, num)
            
    print(len(tensor))
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

    
tensor, tokenizer = tokenize(corpus)

156262
[[   2   50    4 ...    0    0    0]
 [   2   15 2971 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  117 ...    0    0    0]
 [   2  257  195 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fedb64b5f40>


In [6]:
# 단어 사전 어떻게 생겼는지 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성, 마지막 토근은 <end> 가 아니라 <pad> 일 가능성이 높다.
tgt_input = tensor[:, 1:]   # tensor에서 <start>를 잘라내서 타켓 문장을 생성.


In [8]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                         tgt_input,
                                                         train_size=0.8,
                                                         test_size=0.2, random_state=20)

In [9]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (125009, 14)
Target Train: (125009, 14)


-> 학습데이터 갯수가 124960개 이하이므로 이대로 학습을 시켜주겠습니다.

## 평가 데이터셋 만들기

In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_spoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 15000개와, 여기 포함되지 않은 0:<pad>를 포함하여 15001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## 인공지능 만들기

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.drop  = tf.keras.layers.Dropout(0.5)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.drop(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.77126582e-04,  8.56576935e-06,  1.04353858e-04, ...,
         -7.55038272e-05, -7.78922622e-05,  1.04565450e-04],
        [ 4.22573765e-04,  1.34169051e-04,  2.45785108e-04, ...,
         -8.05161544e-05,  4.31105436e-05,  3.08106217e-04],
        [ 7.22432393e-04,  5.35010302e-04,  2.07251884e-04, ...,
         -7.19123927e-05,  1.20504294e-04,  3.07239214e-04],
        ...,
        [ 1.21211470e-03,  2.35255482e-03, -6.85962499e-04, ...,
         -4.46773454e-04,  4.33438458e-04, -9.34198906e-04],
        [ 1.31492992e-03,  2.56342394e-03, -8.33245984e-04, ...,
         -7.74964807e-04,  7.04416190e-04, -1.16331817e-03],
        [ 1.41516177e-03,  2.76693143e-03, -9.39389749e-04, ...,
         -1.12283556e-03,  9.77909891e-04, -1.37121684e-03]],

       [[ 1.77126582e-04,  8.56576935e-06,  1.04353858e-04, ...,
         -7.55038272e-05, -7.78922622e-05,  1.04565450e-04],
        [ 2.32334831e-04, -1.64986932e-06,  1

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  18882560  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 80,107,489
Trainable params: 80,107,489
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
610/610 [==============================] - 601s 981ms/step - loss: 3.2773
Epoch 2/10
610/610 [==============================] - 593s 971ms/step - loss: 2.8082
Epoch 3/10
610/610 [==============================] - 601s 985ms/step - loss: 2.5593
Epoch 4/10


In [ ]:
results = model.evaluate(enc_val,  dec_val, verbose=2)

print(results)

평가 문항 상세 기준

1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?
    - 텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?
    
2. 데이터의 전처리와 데이터셋의 구성 과정이 체계적으로 진행되었는가?
    - 특수문제 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?
    
3. 텍스트 생성모델이 안정적으로 학습되엇는가?
    - 텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?